In [1]:
import time
import numpy as np
import yastn
import yastn.tn.mps as mps
import yastn.tn.fpeps as peps
from routines import NSpin12, MapHex127_r4, MapHex127_r3, gates_from_HH, gates_from_HH2, measure_H_ctm, measure_H_mps

In [2]:
fname = "/home/marek/hex/problems/transfer_Hamiltonian_ibm_kyiv_0_angles_16_0_p_20.pkl"
data = np.load(fname, allow_pickle=True)

In [3]:
# Hamiltonian
fname = "./problems/sherbrooke0v3.pkl"
data = np.load(fname, allow_pickle=True)
HC = data["H"]
HM = {(n,): 1 for n in range(127)}
#
angles = {"p=3": {"beta": (0.50502, 0.35713, 0.19264),
                  "gamma": (-0.14264, -0.26589, -0.34195)},
          "p=4": {"beta": (0.54321, 0.41806, 0.28615, 0.16041),
                  "gamma": (-0.12077, -0.22360, -0.29902, -0.35329)},
          "p=5": {"beta": (0.53822, 0.44776, 0.32923, 0.23056, 0.12587),
                  "gamma": (-0.11764, -0.19946, -0.268736, -0.321586, -0.34583)},}

In [4]:
# group spins
hex4 = MapHex127_r4()
ops = NSpin12(sym='dense')
#
# translate Hamiltonian to grouped sites
HHC4 = hex4.group_H(HC, ops.z, ops.I)
HHM4 = hex4.group_H(HM, ops.x, ops.I)

In [7]:
g1 = gates_from_HH(HHC4, 1)
g2 = gates_from_HH2(HHC4, 1)

In [8]:
len(g2.nn)

45

In [9]:
problem = "p=5"
D = 8  # PEPS bond dimension
#
betas = angles[problem]['beta']
gammas = angles[problem]['gamma']
#
# initial product state
#
geometry = peps.SquareLattice(dims=hex4.dims, boundary='obc')
vectors = {site: ops.vec_x((1,) * Nloc) for site, Nloc in hex4.s2N.items()}
psi4 = peps.product_peps(geometry, vectors)
#
#  run evolution
#
opts_svd_evol = {"D_total": D, "tol": 1e-10}
env4 = peps.EnvNTU(psi4, which='NN+')
#
infoss = []
t0 = time.time()
for step, (gamma, beta) in enumerate(zip(gammas, betas)):
    gates = gates_from_HH2(HHC4, gamma)
    infos = peps.evolution_step_(env4, gates, opts_svd=opts_svd_evol, symmetrize=False)
    infoss.append(infos)
    gates = gates_from_HH2(HHM4, beta)
    infos = peps.evolution_step_(env4, gates, opts_svd=opts_svd_evol, symmetrize=False)
    Delta = peps.accumulated_truncation_error(infoss, statistics='mean')
    print(f"Step: {step}; Evolution time: {time.time() - t0:3.2f}; Truncation error: {Delta:0.6f}")

Step: 0; Evolution time: 8.77; Truncation error: 0.000000
Step: 1; Evolution time: 18.04; Truncation error: 0.000000
Step: 2; Evolution time: 28.00; Truncation error: 0.000000
Step: 3; Evolution time: 52.77; Truncation error: 0.005804
Step: 4; Evolution time: 80.96; Truncation error: 0.019145


In [10]:
opts_svd = {"D_total": D}
env4_ctm = peps.EnvCTM(psi4, init='eye')
t0 = time.time()
for info in env4_ctm.ctmrg_(max_sweeps=10, opts_svd=opts_svd, corner_tol=1e-5, iterator_step=1, method='2site'): #
    print("Ctmrg time:", time.time() - t0)
    print(info)

Ctmrg time: 1.766716480255127
CTMRG_out(sweeps=1, max_dsv=nan, converged=False, max_D=8)
Ctmrg time: 23.074958324432373
CTMRG_out(sweeps=2, max_dsv=0.6624171271641824, converged=False, max_D=8)
Ctmrg time: 45.10183572769165
CTMRG_out(sweeps=3, max_dsv=0.0280115030023803, converged=False, max_D=8)
Ctmrg time: 66.8723361492157
CTMRG_out(sweeps=4, max_dsv=0.0005280590610317658, converged=False, max_D=8)
Ctmrg time: 87.85365891456604
CTMRG_out(sweeps=5, max_dsv=5.409021270875622e-06, converged=True, max_D=8)


In [36]:
env4_BP = peps.EnvBP(psi4)
t0 = time.time()
for info in env4_BP.iterate_(max_sweeps=10, iterator_step=1): #  corner_tol=1e-5,
    print("Ctmrg time:", time.time() - t0)
    print(info)

Ctmrg time: 0.45688509941101074
BP_out(sweeps=1, max_diff=np.float64(1.0121205834468923), converged=None)
Ctmrg time: 0.9782843589782715
BP_out(sweeps=2, max_diff=np.float64(0.39112019074261967), converged=None)
Ctmrg time: 1.4596245288848877
BP_out(sweeps=3, max_diff=np.float64(0.01628378160186533), converged=None)
Ctmrg time: 1.9591248035430908
BP_out(sweeps=4, max_diff=np.float64(0.0003889551523375502), converged=None)
Ctmrg time: 2.7113311290740967
BP_out(sweeps=5, max_diff=np.float64(7.2261843852377706e-06), converged=None)
Ctmrg time: 3.218902587890625
BP_out(sweeps=6, max_diff=np.float64(4.2721706174664415e-07), converged=None)
Ctmrg time: 3.682129383087158
BP_out(sweeps=7, max_diff=np.float64(3.469142316479831e-09), converged=None)
Ctmrg time: 4.117254734039307
BP_out(sweeps=8, max_diff=np.float64(2.7457865089129868e-11), converged=None)
Ctmrg time: 4.55954122543335
BP_out(sweeps=9, max_diff=np.float64(1.589288906375893e-13), converged=None)
Ctmrg time: 5.33596396446228
BP_out(

In [54]:
eng0, ZZ0 = measure_H_ctm(env4_ctm, HHC4)
print(f"Energy for {problem}: {eng0.real:0.4f}")

Energy for p=5: -145.6594


In [55]:
eng3, ZZ3 = measure_H_ctm(env4_BP, HHC4)
print(f"Energy for {problem}: {eng0.real:0.4f}")

Energy for p=5: -145.6594


In [10]:
env_mps = peps.EnvBoundaryMPS(psi4, opts_svd=opts_svd, setup='lrtb')
eng1, ZZ1 = measure_H_mps(env_mps, HHC4)
print(f"Energy for {problem}: {eng1.real:0.4f}")


Energy for p=3: -125.3721


In [12]:
max(abs(ZZ0[k] - ZZ1[k]) for k in ZZ0)

np.float64(3.5933787266406725e-05)

In [4]:
# group spins
hex3 = MapHex127_r3()
ops = NSpin12(sym='dense')
#
# translate Hamiltonian to grouped sites
HHC3 = hex3.group_H(HC, ops.z, ops.I)
HHM3 = hex3.group_H(HM, ops.x, ops.I)

In [5]:
problem = "p=3"
D = 4 # PEPS bond dimension
#
betas = angles[problem]['beta']
gammas = angles[problem]['gamma']
#
# initial product state
#
geometry = peps.SquareLattice(dims=hex3.dims, boundary='obc')
vectors = {site: ops.vec_x((1,) * Nloc) for site, Nloc in hex3.s2N.items()}
psi3 = peps.product_peps(geometry, vectors)


In [6]:
#
#  run evolution
#
opts_svd_evol = {"D_total": D, "tol": 1e-10}
env3 = peps.EnvBP(psi3, which='NN+BP')
#
infoss = []
t0 = time.time()
for step, (gamma, beta) in enumerate(zip(gammas, betas)):
    gates = gates_from_HH(HHC3, gamma)
    infos = peps.evolution_step_(env3, gates, opts_svd=opts_svd_evol, symmetrize=False)
    infoss.append(infos)
    gates = gates_from_HH(HHM3, beta)
    infos = peps.evolution_step_(env3, gates, opts_svd=opts_svd_evol, symmetrize=False)
    Delta = peps.accumulated_truncation_error(infoss, statistics='mean')
    print(f"Step: {step}; Evolution time: {time.time() - t0:3.2f}; Truncation error: {Delta:0.6f}")

Step: 0; Evolution time: 11.47; Truncation error: 0.000000
Step: 1; Evolution time: 26.18; Truncation error: 0.000000
Step: 2; Evolution time: 44.99; Truncation error: 0.047825


In [7]:
opts_svd = {"D_total":  2 * D}
env_mps = peps.EnvBoundaryMPS(psi3, opts_svd=opts_svd, setup='tlbr')

eng2, ZZ2 = measure_H_mps(env_mps, HHC3)
print(f"Energy for {problem}: {eng2:0.4f}")


Energy for p=3: -126.0892


In [9]:
eng2, ZZ2 = measure_H_mps(env_mps, HHC3)
print(f"Energy for {problem}: {eng2:0.4f}")

Energy for p=3: -126.0892


In [8]:
opts_svd = {"D_total": D}
env3_ctm = peps.EnvCTM(psi3, init='eye')
t0 = time.time()
for info in env3_ctm.ctmrg_(max_sweeps=10, opts_svd=opts_svd, corner_tol=1e-5, iterator_step=1, method='hex'): #
    print("Ctmrg time:", time.time() - t0)
    print(info)

Ctmrg time: 0.4672582149505615
CTMRG_out(sweeps=1, max_dsv=nan, converged=False, max_D=4)
Ctmrg time: 1.1203689575195312
CTMRG_out(sweeps=2, max_dsv=0.6345305288447942, converged=False, max_D=4)
Ctmrg time: 1.8159947395324707
CTMRG_out(sweeps=3, max_dsv=0.3441671333617585, converged=False, max_D=4)
Ctmrg time: 2.562283754348755
CTMRG_out(sweeps=4, max_dsv=0.03873754651148224, converged=False, max_D=4)
Ctmrg time: 3.4664809703826904
CTMRG_out(sweeps=5, max_dsv=0.0045875326221407825, converged=False, max_D=4)
Ctmrg time: 4.149161100387573
CTMRG_out(sweeps=6, max_dsv=1.8889862338900372e-06, converged=True, max_D=4)


In [10]:
eng4, ZZ4 = measure_H_ctm(env3_ctm, HHC3)
print(f"Energy for {problem}: {eng4.real:0.4f}")

Energy for p=3: -126.0889


In [8]:
print(env3_ctm[3, 4].tl.get_shape())
print(env3_ctm[3, 4].bl.get_shape())
print(env3_ctm[3, 4].tr.get_shape())
print(env3_ctm[3, 4].br.get_shape())
print(env3_ctm[3, 4].t.get_shape())
print(env3_ctm[3, 4].b.get_shape())
print(env3_ctm[3, 4].l.get_shape())
print(env3_ctm[3, 4].r.get_shape())
env3_ctm.psi.ket[3, 4].get_shape()

(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 1, 4)
(4, 16, 4)
(4, 16, 4)
(4, 16, 4)


(4, 16, 8)

In [9]:
from itertools import product
inds = {site: [pp for pp in product(*([[-1, 1]] * Nloc))] for site, Nloc in hex3.s2N.items()}
vecs = {site: [ops.vec_z(pp) for pp in ind] for site, ind in inds.items()}
opss = {site: [yastn.tensordot(vv, vv.conj(), axes=((), ())) for vv in vvs] for site, vvs in vecs.items()}
pros = {site: [env3.measure_1site(op, site) for op in ops] for site, ops in opss.items()}


In [10]:
pros = {site: [env3.measure_1site(op, site) for op in ops] for site, ops in opss.items()}

In [13]:
out, probability = env3_ctm.sample(projectors=opss, return_probabilities=True, number=10)

In [14]:
probability

[np.float64(3.9144281465918965e-25),
 np.float64(5.753106045263758e-21),
 np.float64(1.6377032701773157e-26),
 np.float64(8.518025107429701e-26),
 np.float64(4.746158691891664e-25),
 np.float64(9.041426146130556e-25),
 np.float64(5.514766883537307e-25),
 np.float64(5.172223141931077e-27),
 np.float64(6.692770550429099e-21),
 np.float64(1.0798818014551618e-18)]

In [15]:
out

{Site(nx=0, ny=0): [30, 25, 8, 26, 19, 10, 9, 31, 8, 24],
 Site(nx=1, ny=0): [7, 7, 28, 7, 5, 25, 7, 5, 7, 7],
 Site(nx=2, ny=0): [10, 30, 27, 31, 4, 30, 31, 31, 26, 30],
 Site(nx=3, ny=0): [27, 20, 11, 11, 11, 13, 10, 27, 11, 11],
 Site(nx=4, ny=0): [16, 0, 17, 0, 17, 14, 12, 16, 16, 17],
 Site(nx=5, ny=0): [9, 19, 27, 25, 19, 23, 7, 1, 7, 19],
 Site(nx=6, ny=0): [3, 3, 2, 3, 3, 2, 3, 3, 2, 3],
 Site(nx=0, ny=1): [7, 6, 5, 2, 7, 2, 7, 7, 3, 5],
 Site(nx=1, ny=1): [2, 2, 1, 3, 1, 1, 2, 1, 3, 1],
 Site(nx=2, ny=1): [2, 2, 0, 2, 0, 2, 2, 0, 0, 2],
 Site(nx=3, ny=1): [2, 2, 1, 1, 0, 3, 3, 0, 1, 0],
 Site(nx=4, ny=1): [5, 5, 0, 1, 6, 5, 0, 7, 7, 4],
 Site(nx=5, ny=1): [2, 1, 0, 2, 1, 2, 1, 2, 2, 0],
 Site(nx=6, ny=1): [3, 1, 0, 3, 1, 0, 1, 1, 1, 1],
 Site(nx=0, ny=2): [2, 3, 3, 1, 3, 1, 2, 3, 2, 1],
 Site(nx=1, ny=2): [5, 4, 7, 1, 1, 4, 5, 1, 4, 1],
 Site(nx=2, ny=2): [2, 2, 2, 3, 3, 2, 2, 3, 2, 1],
 Site(nx=3, ny=2): [3, 2, 2, 1, 3, 2, 2, 2, 1, 1],
 Site(nx=4, ny=2): [1, 1, 2, 0, 3, 1, 2,

In [16]:
out, probability = env_mps.sample(projectors=opss, return_probabilities=True, number=10)

In [17]:
probability

[np.float64(1.9258949598554826e-25),
 np.float64(1.764227068686928e-25),
 np.float64(3.75861107871388e-26),
 np.float64(4.355521598701734e-24),
 np.float64(1.6188063549216878e-22),
 np.float64(9.300159383001698e-24),
 np.float64(9.598050714619655e-28),
 np.float64(1.325713837306452e-23),
 np.float64(3.973941480920912e-25),
 np.float64(2.1077190255109144e-27)]

In [18]:
out

{Site(nx=0, ny=0): [19, 17, 8, 9, 8, 24, 23, 24, 24, 24],
 Site(nx=1, ny=0): [29, 31, 19, 7, 27, 7, 7, 11, 6, 28],
 Site(nx=2, ny=0): [19, 3, 26, 30, 27, 12, 19, 26, 27, 18],
 Site(nx=3, ny=0): [13, 11, 15, 11, 11, 27, 15, 11, 22, 11],
 Site(nx=4, ny=0): [25, 23, 17, 17, 17, 14, 17, 16, 17, 8],
 Site(nx=5, ny=0): [17, 27, 19, 25, 19, 7, 31, 24, 31, 23],
 Site(nx=6, ny=0): [3, 3, 2, 3, 3, 3, 5, 2, 3, 3],
 Site(nx=0, ny=1): [3, 7, 6, 5, 5, 5, 3, 6, 7, 7],
 Site(nx=1, ny=1): [3, 3, 1, 3, 3, 3, 3, 3, 3, 1],
 Site(nx=2, ny=1): [2, 2, 7, 2, 2, 7, 7, 2, 2, 2],
 Site(nx=3, ny=1): [2, 1, 1, 1, 0, 0, 1, 1, 1, 2],
 Site(nx=4, ny=1): [5, 5, 4, 1, 5, 4, 5, 5, 5, 0],
 Site(nx=5, ny=1): [2, 1, 1, 2, 2, 0, 1, 2, 2, 2],
 Site(nx=6, ny=1): [1, 3, 3, 3, 1, 3, 3, 0, 1, 0],
 Site(nx=0, ny=2): [1, 3, 1, 0, 2, 2, 2, 2, 1, 3],
 Site(nx=1, ny=2): [1, 4, 4, 1, 4, 1, 4, 1, 1, 6],
 Site(nx=2, ny=2): [3, 3, 2, 2, 2, 1, 1, 3, 1, 3],
 Site(nx=3, ny=2): [3, 0, 3, 2, 5, 2, 2, 2, 0, 1],
 Site(nx=4, ny=2): [1, 1, 1, 1, 